# Worksheet 7 - Classification (Part II)

### Lecture and Tutorial Learning Goals:

After completing this week's lecture and tutorial work, you will be able to:

* Describe what a test data set is and how it is used in classification.
* Using Python, evaluate classification accuracy using a test data set and appropriate metrics.
* Using Python, execute cross-validation in Python to choose the number of neighbours.
* Identify when it is necessary to scale variables before classification and do this using Python
* In a dataset with > 2 attributes, perform k-nearest neighbour classification in Python using the `scikit-learn` package to predict the class of a test dataset.
* Describe advantages and disadvantages of the k-nearest neighbour classification algorithm.


In [ ]:
### Run this cell before continuing.
import altair as alt
import numpy as np
import pandas as pd
import sklearn
from sklearn.compose import make_column_transformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

alt.data_transformers.disable_max_rows()
alt.renderers.enable("mimetype")

**Question 0.1** Multiple Choice:
<br>{points: 1}

Before applying k-nearest neighbour to a classification task, we need to scale the data. What is the purpose of this step?

A. To help speed up the knn algorithm.

B. To convert all data observations to numeric values.

C. To ensure all data observations will be on a comparable scale and contribute equal shares to the calculation of the distance between points.

D. None of the above.

*Assign your answer to an object called `answer0_1`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`)*.

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer0_1)).encode("utf-8")+b"130104d8fd908a39").hexdigest() == "335b922fde2572f02af2b844d594c6f4b5d69ce7", "type of answer0_1 is not str. answer0_1 should be an str"
assert sha1(str(len(answer0_1)).encode("utf-8")+b"130104d8fd908a39").hexdigest() == "bde0d331bced8a53c806d716674754e1d6c768f5", "length of answer0_1 is not correct"
assert sha1(str(answer0_1.lower()).encode("utf-8")+b"130104d8fd908a39").hexdigest() == "c5d4e3f95a6af5fc9fe678983799a240a4be1d94", "value of answer0_1 is not correct"
assert sha1(str(answer0_1).encode("utf-8")+b"130104d8fd908a39").hexdigest() == "d6060ab21cefae48c2e9d1871a83bc4aac157efb", "correct string value of answer0_1 but incorrect case of letters"

print('Success!')

## 1. Fruit Data Example - (Part II)
**Question 1.0** 
<br>{points: 1}

First, load the file, `fruit_data.csv` (found in the data folder) from the previous tutorial, into your notebook.

*Assign your data to an object called `fruit_data`.*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_data is None)).encode("utf-8")+b"759c8ec03f4c45c6").hexdigest() == "92c201ba18f7297f7a76cb050822f5ac0436dab1", "type of fruit_data is None is not bool. fruit_data is None should be a bool"
assert sha1(str(fruit_data is None).encode("utf-8")+b"759c8ec03f4c45c6").hexdigest() == "7a486f50ea6b1b071bd9162085df71624a1609e0", "boolean value of fruit_data is None is not correct"

assert sha1(str(type(fruit_data.shape)).encode("utf-8")+b"e383078e14a78be8").hexdigest() == "ebfef3814f286bdbb02eeb30417e3a28a1564615", "type of fruit_data.shape is not tuple. fruit_data.shape should be a tuple"
assert sha1(str(len(fruit_data.shape)).encode("utf-8")+b"e383078e14a78be8").hexdigest() == "a44bfd900e1582a67e23a0ab24aabfbe18be9ede", "length of fruit_data.shape is not correct"
assert sha1(str(sorted(map(str, fruit_data.shape))).encode("utf-8")+b"e383078e14a78be8").hexdigest() == "8253d7c82c35921578271a7e81aacc5e251497bd", "values of fruit_data.shape are not correct"
assert sha1(str(fruit_data.shape).encode("utf-8")+b"e383078e14a78be8").hexdigest() == "4a29724287bb679a524e5cc27b2139f3b872ecc2", "order of elements of fruit_data.shape is not correct"

assert sha1(str(type(fruit_data.fruit_name.dtype)).encode("utf-8")+b"9277c0a680019da4").hexdigest() == "52774b0d7dfa63162862ade134b7a4714713526b", "type of fruit_data.fruit_name.dtype is not correct"
assert sha1(str(fruit_data.fruit_name.dtype).encode("utf-8")+b"9277c0a680019da4").hexdigest() == "980d4ee0e341a066dc856a4d42f6b0f7200e0936", "value of fruit_data.fruit_name.dtype is not correct"

print('Success!')

Let's take a look at the first six observations in the fruit dataset. Run the cell below.

In [ ]:
# Run this cell.
fruit_data.head(6)

Run the cell below, and find the nearest neighbour based on mass and width to the first observation just by looking at the scatterplot (the first observation has been circled for you).

In [ ]:
# Run this cell.
point1 = [192, 8.4]
point2 = [180, 8]
point44 = [194, 7.2]

fruit_chart = (
    alt.Chart(fruit_data)
    .mark_point(size=15)
    .encode(
        x=alt.X("mass", title="Mass (grams)"),
        y=alt.Y("width", title="Width (cm)", scale=alt.Scale(zero=False)),
        color=alt.Color("fruit_name", title="Name of the Fruit"),
    )
)

(
    fruit_chart
    + alt.Chart(pd.DataFrame([[192, 8.4]], columns=["x", "y"]))
    .mark_point(size=150)
    .encode(x="x", y="y", color=alt.value("black"))
    + alt.Chart(pd.DataFrame([[1, 183, 8.5]], columns=["text", "x", "y"]))
    .mark_text(size=15)
    .encode(x="x", y="y", text="text", color=alt.value("black"))
).configure_axis(labelFontSize=20, titleFontSize=20).configure_legend(
    titleFontSize=15, labelFontSize=15
).properties(
    width=400, height=300
)

**Question 1.1** Multiple Choice:
<br>{points: 1}

Based on the graph generated, what is the `fruit_name` of the closest data point to the one circled?

A. apple

B. lemon

C. mandarin

D. orange

*Assign your answer to an object called `answer1_1`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_1)).encode("utf-8")+b"eb9f243b4f0f76c6").hexdigest() == "167e6e762a0011f5cf344eeba6c64b8720539cb4", "type of answer1_1 is not str. answer1_1 should be an str"
assert sha1(str(len(answer1_1)).encode("utf-8")+b"eb9f243b4f0f76c6").hexdigest() == "99bc57427342ad4fa2d8e10e3d1c59684228298e", "length of answer1_1 is not correct"
assert sha1(str(answer1_1.lower()).encode("utf-8")+b"eb9f243b4f0f76c6").hexdigest() == "99cb8f78d53f9815402d838d78ebb5aa98778d3f", "value of answer1_1 is not correct"
assert sha1(str(answer1_1).encode("utf-8")+b"eb9f243b4f0f76c6").hexdigest() == "d72c30335470fb038cb37838c5604ebe766eb8c7", "correct string value of answer1_1 but incorrect case of letters"

print('Success!')

**Question 1.2**
<br>{points: 1}

Using `mass` and `width`, calculate the distance between the first observation and the second observation.

We provide a scaffolding to get you started.

*Assign your answer to an object called `fruit_dist_2`.*

In [ ]:
# ___ = euclidean_distances(fruit_data.loc[0:1, ["mass", ___]])

# your code here
raise NotImplementedError
fruit_dist_2

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_dist_2)).encode("utf-8")+b"82a086154e5a8ca0").hexdigest() == "a38b596c0ed145ce5b0d9f5e0ecc51624b943628", "type of fruit_dist_2 is not correct"
assert sha1(str(fruit_dist_2).encode("utf-8")+b"82a086154e5a8ca0").hexdigest() == "f9c52661734912be39cf4bf6bbb8f26a60d231af", "value of fruit_dist_2 is not correct"

print('Success!')

**Question 1.3**
<br>{points: 1}

Calculate the distance between the first and the **44th observation** in the `fruit` dataset using the `mass` and `width` variables.

You can see from the data frame output from the cell below that **observation 44** has `mass` = 194 g and `width` = 7.2 cm.

*Assign your answer to an object called `fruit_dist_44`.*

In [ ]:
# Run this cell to see the 44th observation
pd.DataFrame(fruit_data.iloc[43, :]).T

In [ ]:
# Run this cell.
point1 = [192, 8.4]
point2 = [180, 8]
point44 = [194, 7.2]

(
    fruit_chart
    + alt.Chart(
        pd.DataFrame([[192, 8.4], [180, 8.0], [193.5, 7.2]], columns=["x", "y"])
    )
    .mark_point(size=150)
    .encode(x="x", y="y", color=alt.value("black"))
    + alt.Chart(
        pd.DataFrame(
            [[1, 183, 8.5], [2, 169, 8.1], [44, 204, 7.1]], columns=["text", "x", "y"]
        )
    )
    .mark_text(size=15)
    .encode(x="x", y="y", text="text", color=alt.value("black"))
).configure_axis(labelFontSize=20, titleFontSize=20).configure_legend(
    titleFontSize=15, labelFontSize=15
).properties(
    width=400, height=300
)

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_dist_44)).encode("utf-8")+b"6234662a41641052").hexdigest() == "4e09468cf2f8681eac4a737c4edadb2a06118f08", "type of fruit_dist_44 is not correct"
assert sha1(str(fruit_dist_44).encode("utf-8")+b"6234662a41641052").hexdigest() == "c618ce50ccadcf7e786d2a14eb5381e1f828acb9", "value of fruit_dist_44 is not correct"

print('Success!')

What do you notice about your answers from **Question 1.2** & **1.3** that you just calculated? Is it what you would expect given the scatter plot above? Why or why not? Discuss about this.

*Hint: Look at where the observations are on the scatterplot in the cell above this question, and what might happen if we changed grams into kilograms to measure the mass?*

**Question 1.4**
<br>{points: 1}

From the distance calculation, we see that observation 1 and 44 have a smaller distance than observation 1 and 2. However, if we look at the scatterplot the distance of the first observation to the second observation appears closer than to the 44th observation.

Which of the following statements is correct?

A. A difference of 12 g in mass between observation 1 and 2 is large compared to a difference of 1.2 cm in width between observation 1 and 44. Consequently, mass will drive the classification results, and width will have less of an effect. Hence, our distance calculation reflects that.

B. If we measured mass in kilograms, then we’d get different classification results.

C. We should standardize the data so that all variables will be on a comparable scale.

D. All of the above.

*Assign your answer to an object called `answer1_4`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_4)).encode("utf-8")+b"488e228983b53ac8").hexdigest() == "36d20f2fb599c4d32ac3236f5aa4eb296025e311", "type of answer1_4 is not str. answer1_4 should be an str"
assert sha1(str(len(answer1_4)).encode("utf-8")+b"488e228983b53ac8").hexdigest() == "7bade95e88fcf6588819978cb12a6db2775f26d1", "length of answer1_4 is not correct"
assert sha1(str(answer1_4.lower()).encode("utf-8")+b"488e228983b53ac8").hexdigest() == "35a6d18527f3318e704741aebb5f953ac16da3c9", "value of answer1_4 is not correct"
assert sha1(str(answer1_4).encode("utf-8")+b"488e228983b53ac8").hexdigest() == "169e1d85ea58bb7ad834e427de7da7cc1d564db0", "correct string value of answer1_4 but incorrect case of letters"

print('Success!')

**Question 1.5**
<br>{points: 1}

Scale and center all the variables of the `fruit` dataset and save them as columns in your data table. We will use the `StandardScaler` in the preprocessor. Then `.fit_transform` the preprocessor so that we can examine the output.

Fit and transform your preprocessor with predictors `mass`, `width`, `height`, and `color_score`. For other columns, we will `passthrough` them in the preprocessor.

Name the preprocessor `fruit_data_preprocessor`. Concatenate the transformed columns with the original dataframe and save the dataset object in a dataframe and call it `fruit_data_scaled`. Make sure to name the new columns `scaled_*` where * is the old column name (e.g. `scaled_mass`). 

In [ ]:
# ___ = make_column_transformer(
#     (
#         "passthrough",
#         [
#             "fruit_label",
#             "fruit_name",
#             "fruit_subtype",
#         ],
#     ),
#     (___, [___, ___, ___, ___]),
# )

# ___ = pd.DataFrame(
#     fruit_data_preprocessor.___(___),
#     columns=[
#         "fruit_label",
#         "fruit_name",
#         "fruit_subtype",
#         ___,
#         ___,
#         ___,
#         ___,
#     ],
# )


# your code here
raise NotImplementedError
fruit_data_scaled.head()

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_data_scaled is None)).encode("utf-8")+b"d760a7474789aac7").hexdigest() == "23b67c42aa7f5a5c72a67e130f5b70110e17a222", "type of fruit_data_scaled is None is not bool. fruit_data_scaled is None should be a bool"
assert sha1(str(fruit_data_scaled is None).encode("utf-8")+b"d760a7474789aac7").hexdigest() == "b2856f3167acc0cb2b8bc5dcc2737b481633cf71", "boolean value of fruit_data_scaled is None is not correct"

assert sha1(str(type(fruit_data_scaled.fruit_name.dtype)).encode("utf-8")+b"c30db3c98af03a39").hexdigest() == "41d77ed6b53d04559bb5e379db348081361ad2bd", "type of fruit_data_scaled.fruit_name.dtype is not correct"
assert sha1(str(fruit_data_scaled.fruit_name.dtype).encode("utf-8")+b"c30db3c98af03a39").hexdigest() == "a883181d291abcb046fa1672fb280ea325083d2b", "value of fruit_data_scaled.fruit_name.dtype is not correct"

assert sha1(str(type(fruit_data_preprocessor.transformers_[0][2])).encode("utf-8")+b"d03ecfc41ba1e82c").hexdigest() == "9e452600278e66fbd13f96ea1e4bf51bcf2f7645", "type of fruit_data_preprocessor.transformers_[0][2] is not list. fruit_data_preprocessor.transformers_[0][2] should be a list"
assert sha1(str(len(fruit_data_preprocessor.transformers_[0][2])).encode("utf-8")+b"d03ecfc41ba1e82c").hexdigest() == "f63f942b26f17bb897d6b3298406d1ddf00f54cd", "length of fruit_data_preprocessor.transformers_[0][2] is not correct"
assert sha1(str(sorted(map(str, fruit_data_preprocessor.transformers_[0][2]))).encode("utf-8")+b"d03ecfc41ba1e82c").hexdigest() == "a8e5f6892e3bb09b7c97e56e5f92c918dcdcdd60", "values of fruit_data_preprocessor.transformers_[0][2] are not correct"
assert sha1(str(fruit_data_preprocessor.transformers_[0][2]).encode("utf-8")+b"d03ecfc41ba1e82c").hexdigest() == "a8e5f6892e3bb09b7c97e56e5f92c918dcdcdd60", "order of elements of fruit_data_preprocessor.transformers_[0][2] is not correct"

assert sha1(str(type(fruit_data_preprocessor.transformers_[1][1])).encode("utf-8")+b"9e9d23fb1a93427a").hexdigest() == "ddc688ee0ee92a2e8762d499cc15d97307df1485", "type of fruit_data_preprocessor.transformers_[1][1] is not correct"
assert sha1(str(fruit_data_preprocessor.transformers_[1][1]).encode("utf-8")+b"9e9d23fb1a93427a").hexdigest() == "1a17674522d22dcafa93fce0ca8b794204afdf91", "value of fruit_data_preprocessor.transformers_[1][1] is not correct"

assert sha1(str(type(fruit_data_preprocessor.transformers_[1][2])).encode("utf-8")+b"75efd27efcd736c6").hexdigest() == "e97c6b89c4af0e014e96ca38d5e26cbe13223b88", "type of fruit_data_preprocessor.transformers_[1][2] is not list. fruit_data_preprocessor.transformers_[1][2] should be a list"
assert sha1(str(len(fruit_data_preprocessor.transformers_[1][2])).encode("utf-8")+b"75efd27efcd736c6").hexdigest() == "353abd001ac64b924765438d8128f74341c8410e", "length of fruit_data_preprocessor.transformers_[1][2] is not correct"
assert sha1(str(sorted(map(str, fruit_data_preprocessor.transformers_[1][2]))).encode("utf-8")+b"75efd27efcd736c6").hexdigest() == "44e20f84f8c68178c1da6e3874c25af280483bfb", "values of fruit_data_preprocessor.transformers_[1][2] are not correct"
assert sha1(str(fruit_data_preprocessor.transformers_[1][2]).encode("utf-8")+b"75efd27efcd736c6").hexdigest() == "f88d1c5fd82ecaa6203bb83af9d47b8e48f2b199", "order of elements of fruit_data_preprocessor.transformers_[1][2] is not correct"

assert sha1(str(type(sum(fruit_data_scaled.scaled_mass.dropna()))).encode("utf-8")+b"2dcd6efea813c7ff").hexdigest() == "c20a16fa233d5b86790de687bb8b788ce3258e6d", "type of sum(fruit_data_scaled.scaled_mass.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(fruit_data_scaled.scaled_mass.dropna()), 2)).encode("utf-8")+b"2dcd6efea813c7ff").hexdigest() == "518ff2a27dfa5bd98b83e4378a233bc767fce78b", "value of sum(fruit_data_scaled.scaled_mass.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(fruit_data_scaled.scaled_width.dropna()))).encode("utf-8")+b"4dd344661128f6a3").hexdigest() == "c6b26a813141eedb52101824efa99cee4a8d1415", "type of sum(fruit_data_scaled.scaled_width.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(fruit_data_scaled.scaled_width.dropna()), 2)).encode("utf-8")+b"4dd344661128f6a3").hexdigest() == "e00d56e9f16422d1339abd3040d2fc4d7549886f", "value of sum(fruit_data_scaled.scaled_width.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(fruit_data_scaled.scaled_height.dropna()))).encode("utf-8")+b"ed72356454c2bf83").hexdigest() == "aacdd051f4504527854028c09806f318d15e543e", "type of sum(fruit_data_scaled.scaled_height.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(fruit_data_scaled.scaled_height.dropna()), 2)).encode("utf-8")+b"ed72356454c2bf83").hexdigest() == "ddd4bb9142d664b3e94829bf0e6c41433f4005ca", "value of sum(fruit_data_scaled.scaled_height.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(fruit_data_scaled.scaled_color_score.dropna()))).encode("utf-8")+b"c29cbff8c01ee1ff").hexdigest() == "0d41d3766b1bba5556214dc52b8eb28ddf8a0501", "type of sum(fruit_data_scaled.scaled_color_score.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(fruit_data_scaled.scaled_color_score.dropna()), 2)).encode("utf-8")+b"c29cbff8c01ee1ff").hexdigest() == "ca9d9088884a58379ebb20eb9804830eec44ada8", "value of sum(fruit_data_scaled.scaled_color_score.dropna()) is not correct (rounded to 2 decimal places)"

print('Success!')


**Question 1.6**
<br>{points: 1}

Let's repeat **Question 1.2** and **1.3** with the scaled variables:

- calculate the distance with the scaled mass and width variables between observations 1 and 2
- calculate the distances with the scaled mass and width variables between observations 1 and 44

After you do this, think about how these distances compared to the distances you computed in **Question 1.2** and **1.3** for the same points.

*Assign your answers to objects called `distance_2` and `distance_44` respectively.*

In [ ]:
# your code here
raise NotImplementedError
print(distance_2)
print(distance_44)

In [ ]:
from hashlib import sha1
assert sha1(str(type(distance_2 is None)).encode("utf-8")+b"35b0eefa0278158e").hexdigest() == "811d5953234c799e6e4eb897762d2e9f0e235acc", "type of distance_2 is None is not bool. distance_2 is None should be a bool"
assert sha1(str(distance_2 is None).encode("utf-8")+b"35b0eefa0278158e").hexdigest() == "e7f09a243f461d0d53874c9ef3785db2303fb3d1", "boolean value of distance_2 is None is not correct"

assert sha1(str(type(distance_2)).encode("utf-8")+b"ec33b937be17807b").hexdigest() == "cc1fcce571976c9d8a73c22eadf91ec23586600c", "type of type(distance_2) is not correct"

assert sha1(str(type(distance_2)).encode("utf-8")+b"749aaeb73fad1ad9").hexdigest() == "2b41a47ac340fb01c438777dd81e2fc93e3f201a", "type of distance_2 is not correct"
assert sha1(str(distance_2).encode("utf-8")+b"749aaeb73fad1ad9").hexdigest() == "496424843d0f1eeac355647be16e3b5f420dc1c5", "value of distance_2 is not correct"

assert sha1(str(type(distance_44 is None)).encode("utf-8")+b"eefd5c895362067d").hexdigest() == "acbda19d20dac048987108c42b79ed550b85d29a", "type of distance_44 is None is not bool. distance_44 is None should be a bool"
assert sha1(str(distance_44 is None).encode("utf-8")+b"eefd5c895362067d").hexdigest() == "2f6ae2c7bf30a4bb968ac32798986c28eb057f5e", "boolean value of distance_44 is None is not correct"

assert sha1(str(type(distance_44)).encode("utf-8")+b"ccb4ed8890d289a0").hexdigest() == "34913df117768ba1ed3a9fe5c3625b4734faf902", "type of type(distance_44) is not correct"

assert sha1(str(type(distance_44)).encode("utf-8")+b"16e09764b92b347f").hexdigest() == "5476a068ea922ba8468cab9b3ecd122245a1d9cf", "type of distance_44 is not correct"
assert sha1(str(distance_44).encode("utf-8")+b"16e09764b92b347f").hexdigest() == "d12ce22fccc53136329aa6883700ef976ec0c188", "value of distance_44 is not correct"

print('Success!')

## Randomness and Setting Seeds

This worksheet uses functions from the `scikit-learn` library, which not only allows us to perform K-nearest neighbour classification, but also allows us to evaluate how well our classification worked. In order to ensure that the steps in the worksheet are reproducible, we need to set a *`random_state`* or *random seed*, i.e., a numerical "starting value," which determines the sequence of random numbers Python will generate.

Below in many cells we have included an argument to set the `random_state` or `np.random.seed`. They are necessary to make sure the autotesting code functions properly.

## 2. Splitting the data into a training and test set

In this exercise, we will be partitioning `fruit_data` into a training (75%) and testing (25%) set using the `scikit-learn` package. After creating the test set, we will put the test set away in a lock box and not touch it again until we have found the best k-nn classifier we can make using the training set. We will use the variable `fruit_name` as our class label. 


**Question 2.0**
<br> {points: 1}

To create the training and test set, we would use the `train_test_split` function from `scikit-learn` pacakge. Save the trained dataset and test dataset as `fruit_train` and `fruit_test`, respectively. 

In [ ]:
# Randomly take 75% of the data in the training set.
# This will be proportional to the different number of fruit names in the dataset.

# ___, ___ = train_test_split(___, test_size=___, random_state=123) # set the random state to be 123

# your code here
raise NotImplementedError
print(fruit_train.head())
print(fruit_test.head())

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_train is None)).encode("utf-8")+b"6a424a941b4d33d1").hexdigest() == "3be500abb0d6494fb4646f0da1b4084690c81cb2", "type of fruit_train is None is not bool. fruit_train is None should be a bool"
assert sha1(str(fruit_train is None).encode("utf-8")+b"6a424a941b4d33d1").hexdigest() == "eb9e245c293e6d97ea3b05313c5b4d63e3a43682", "boolean value of fruit_train is None is not correct"

assert sha1(str(type(fruit_test is None)).encode("utf-8")+b"561a71f730122439").hexdigest() == "d3a4d80b21a49d3fdd5047999c74a9947b86a99f", "type of fruit_test is None is not bool. fruit_test is None should be a bool"
assert sha1(str(fruit_test is None).encode("utf-8")+b"561a71f730122439").hexdigest() == "09a6ab80437d288ef8fa47f69657123c0a47c122", "boolean value of fruit_test is None is not correct"

assert sha1(str(type(fruit_train.shape)).encode("utf-8")+b"e21997d35337b71e").hexdigest() == "032bb0ca96c3cb0b85ed00ee12030dc6f5a42d7f", "type of fruit_train.shape is not tuple. fruit_train.shape should be a tuple"
assert sha1(str(len(fruit_train.shape)).encode("utf-8")+b"e21997d35337b71e").hexdigest() == "23efecf6b35e5f7714ce98613a93b3a3a61e0226", "length of fruit_train.shape is not correct"
assert sha1(str(sorted(map(str, fruit_train.shape))).encode("utf-8")+b"e21997d35337b71e").hexdigest() == "e6f5b35f886cd6e1fa736fd226a1de76ab775311", "values of fruit_train.shape are not correct"
assert sha1(str(fruit_train.shape).encode("utf-8")+b"e21997d35337b71e").hexdigest() == "60b467e71eee3b8aa48a10c14cf630df6f9333fc", "order of elements of fruit_train.shape is not correct"

assert sha1(str(type(fruit_test.shape)).encode("utf-8")+b"623d0c21aa34c4a7").hexdigest() == "04d2e838259ba562492686d6e89fac9bc243e4b6", "type of fruit_test.shape is not tuple. fruit_test.shape should be a tuple"
assert sha1(str(len(fruit_test.shape)).encode("utf-8")+b"623d0c21aa34c4a7").hexdigest() == "10070e1093771575b5868271255086122d33e016", "length of fruit_test.shape is not correct"
assert sha1(str(sorted(map(str, fruit_test.shape))).encode("utf-8")+b"623d0c21aa34c4a7").hexdigest() == "f0e797865417e96ba4282dc496c0e9ff7ed139a2", "values of fruit_test.shape are not correct"
assert sha1(str(fruit_test.shape).encode("utf-8")+b"623d0c21aa34c4a7").hexdigest() == "359cc98563050bcac04aa64d05b517ab0d36fd95", "order of elements of fruit_test.shape is not correct"

assert sha1(str(type(sum(fruit_train.mass))).encode("utf-8")+b"baeb7739b03af0ee").hexdigest() == "95a4fd65f977171d4c0e56d47ce020c2731a3473", "type of sum(fruit_train.mass) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(fruit_train.mass)).encode("utf-8")+b"baeb7739b03af0ee").hexdigest() == "5444650eb3f419734f3750cfa3d85f6a1d7ab067", "value of sum(fruit_train.mass) is not correct"

assert sha1(str(type(sum(fruit_test.mass))).encode("utf-8")+b"d64d2fa142f18218").hexdigest() == "c464520a8f71aeb692363fa76c8071e7a984ab08", "type of sum(fruit_test.mass) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(fruit_test.mass)).encode("utf-8")+b"d64d2fa142f18218").hexdigest() == "548102dda43c692ad1136fb0fe73a0cd338c39c8", "value of sum(fruit_test.mass) is not correct"

print('Success!')

**Question 2.1** 
<br> {points: 1}

K-nearest neighbors is sensitive to the scale of the predictors so we should do some preprocessing to standardize them. Remember that standardizing involves centering/shifting (subtracting the mean of each variable) and scaling (dividing by its standard deviation). Also remember that standardization is *part of your training procedure*, so you can't use your test data to compute the shift / scale values for each variable. Therefore, you must pass only the training data to your preprocessor to compute the preprocessing steps. This ensures that our test data does not influence any aspect of our model training. Once we have created the standardization preprocessor, we can then later on apply it separately to both the training and test data sets.

For this exercise, let's see if `mass` and `color_score` can predict `fruit_name`. 

To scale and center the data, first, pass the predictors to the `make_column_transformer` function to make the preprocessor.

*Assign your answer to an object called `fruit_preprocessor`.*

In [ ]:
# ___ = make_column_transformer(
#     (___, [___, ___]),
# )

# your code here
raise NotImplementedError
fruit_preprocessor

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_preprocessor is None)).encode("utf-8")+b"749fb45c444cc04d").hexdigest() == "7292453ac1685d0c548a4ec567dcab2a517e2a30", "type of fruit_preprocessor is None is not bool. fruit_preprocessor is None should be a bool"
assert sha1(str(fruit_preprocessor is None).encode("utf-8")+b"749fb45c444cc04d").hexdigest() == "c30c6849facceb99906250e1a55e049cb7170722", "boolean value of fruit_preprocessor is None is not correct"

assert sha1(str(type(type(fruit_preprocessor))).encode("utf-8")+b"0c48525d8f72a62b").hexdigest() == "321a3fc0d6cec4954934990450f3478c9e985b1b", "type of type(fruit_preprocessor) is not correct"
assert sha1(str(type(fruit_preprocessor)).encode("utf-8")+b"0c48525d8f72a62b").hexdigest() == "e965d94728fbbc7fe0a1c7519e37acc3262f8bf9", "value of type(fruit_preprocessor) is not correct"

assert sha1(str(type(fruit_preprocessor.transformers[0][0])).encode("utf-8")+b"69ea643af35c33ff").hexdigest() == "a9e257738be75ae9f7b3a8798bd95ac794286f1c", "type of fruit_preprocessor.transformers[0][0] is not str. fruit_preprocessor.transformers[0][0] should be an str"
assert sha1(str(len(fruit_preprocessor.transformers[0][0])).encode("utf-8")+b"69ea643af35c33ff").hexdigest() == "01e03a83308d6d9c8c02280f87df89d5872ca9bb", "length of fruit_preprocessor.transformers[0][0] is not correct"
assert sha1(str(fruit_preprocessor.transformers[0][0].lower()).encode("utf-8")+b"69ea643af35c33ff").hexdigest() == "e9ea99a7778b246d4d6400d1daed0e04359b21ea", "value of fruit_preprocessor.transformers[0][0] is not correct"
assert sha1(str(fruit_preprocessor.transformers[0][0]).encode("utf-8")+b"69ea643af35c33ff").hexdigest() == "e9ea99a7778b246d4d6400d1daed0e04359b21ea", "correct string value of fruit_preprocessor.transformers[0][0] but incorrect case of letters"

assert sha1(str(type(fruit_preprocessor.transformers[0][2])).encode("utf-8")+b"3270f870c020cf6e").hexdigest() == "fe387014c75c6eeb88736d3e59a8e153b21a78e4", "type of fruit_preprocessor.transformers[0][2] is not list. fruit_preprocessor.transformers[0][2] should be a list"
assert sha1(str(len(fruit_preprocessor.transformers[0][2])).encode("utf-8")+b"3270f870c020cf6e").hexdigest() == "4705bfe6072de0e14af8ade8fbd901a047dfc4bf", "length of fruit_preprocessor.transformers[0][2] is not correct"
assert sha1(str(sorted(map(str, fruit_preprocessor.transformers[0][2]))).encode("utf-8")+b"3270f870c020cf6e").hexdigest() == "77c4a5b32a1e03d7b0b84f59a9b3fa1e00d65043", "values of fruit_preprocessor.transformers[0][2] are not correct"
assert sha1(str(fruit_preprocessor.transformers[0][2]).encode("utf-8")+b"3270f870c020cf6e").hexdigest() == "cbd63a8e95612425057126514974119b8a64ca05", "order of elements of fruit_preprocessor.transformers[0][2] is not correct"

print('Success!')

**Question 2.2**
<br> {points: 1}

So far, we have split the training and testing datasets as well as preprocessed the data. Now, let's create our K-nearest neighbour classifier with only the training set using the `scikit-learn` package. First, create the classifier by specifying that we want $K = 3$ neighbors and $weights = "distance"$. *Assign your answer to an object called `knn_spec`*. 

Name the predictor as `X` and the target as `y`. 

Next, train the classifier with the training data set using the `make_pipeline` and `fit` function. The `make_pipeline` function allows you to bundle together your pre-processing, modeling, and post-processing requests. Scaffolding is provided below for you.

*Assign your answer to an object called `fruit_fit`*.

In [ ]:
# ___ = KNeighborsClassifier(n_neighbors=___, weights="distance")

# ___ = ___[["mass", "color_score"]]
# ___ = fruit_train[___]

# ___ = make_pipeline(___, ___).fit(___, ___)

# your code here
raise NotImplementedError
fruit_fit

In [ ]:
from hashlib import sha1
assert sha1(str(type(knn_spec is None)).encode("utf-8")+b"159fc43987ce64bc").hexdigest() == "f1f0e67c8a20c9e86cb824eacdb49d48c01f6595", "type of knn_spec is None is not bool. knn_spec is None should be a bool"
assert sha1(str(knn_spec is None).encode("utf-8")+b"159fc43987ce64bc").hexdigest() == "ba8a718f472dacf65d1cd35da4ebb49c72a65440", "boolean value of knn_spec is None is not correct"

assert sha1(str(type(type(knn_spec))).encode("utf-8")+b"99a9b5483278bd92").hexdigest() == "80413e30f8e0c75adf4473c1deaea7c6b399c7fe", "type of type(knn_spec) is not correct"
assert sha1(str(type(knn_spec)).encode("utf-8")+b"99a9b5483278bd92").hexdigest() == "6a95041877b3a3944000222c5d76cfd33788eb6f", "value of type(knn_spec) is not correct"

assert sha1(str(type(knn_spec.effective_metric_)).encode("utf-8")+b"c8aa1437efbefbb2").hexdigest() == "139dd47bb2d4619c3491e4aed53d26fdb399a84c", "type of knn_spec.effective_metric_ is not str. knn_spec.effective_metric_ should be an str"
assert sha1(str(len(knn_spec.effective_metric_)).encode("utf-8")+b"c8aa1437efbefbb2").hexdigest() == "f0daec6a082b6b831bdad313a990904b5549f52a", "length of knn_spec.effective_metric_ is not correct"
assert sha1(str(knn_spec.effective_metric_.lower()).encode("utf-8")+b"c8aa1437efbefbb2").hexdigest() == "b48503efb237901e20631756e56e107e278a62b7", "value of knn_spec.effective_metric_ is not correct"
assert sha1(str(knn_spec.effective_metric_).encode("utf-8")+b"c8aa1437efbefbb2").hexdigest() == "b48503efb237901e20631756e56e107e278a62b7", "correct string value of knn_spec.effective_metric_ but incorrect case of letters"

assert sha1(str(type(knn_spec.n_neighbors)).encode("utf-8")+b"fd9fe96b62c37aff").hexdigest() == "354d0a32a63c70f183594a008107f972e84223c2", "type of knn_spec.n_neighbors is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(knn_spec.n_neighbors).encode("utf-8")+b"fd9fe96b62c37aff").hexdigest() == "ac15de3656eb41a896ae1bb7a0d4f1ec45e43e36", "value of knn_spec.n_neighbors is not correct"

assert sha1(str(type(sum(X.mass))).encode("utf-8")+b"436656edfbb7cc05").hexdigest() == "dcab74543015842c7e27a9adecef3bb38dcd379c", "type of sum(X.mass) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(X.mass)).encode("utf-8")+b"436656edfbb7cc05").hexdigest() == "7e760810551561507f378aec3deda9481f6cb724", "value of sum(X.mass) is not correct"

assert sha1(str(type(sum(X.color_score))).encode("utf-8")+b"3a341121ed4bedb5").hexdigest() == "1d93e771cfcdd5b2bfbb6aaae9d683410d3dc1cb", "type of sum(X.color_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(X.color_score), 2)).encode("utf-8")+b"3a341121ed4bedb5").hexdigest() == "d64320c197dc7c4c8d629b02f0ff855783160be9", "value of sum(X.color_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(y.name)).encode("utf-8")+b"4d5b290dc60778af").hexdigest() == "4bbed89a035475465ae47cd526c55b0f31828af7", "type of y.name is not str. y.name should be an str"
assert sha1(str(len(y.name)).encode("utf-8")+b"4d5b290dc60778af").hexdigest() == "165c5d0abf580b83c86afd312cecdbd7d8055e1a", "length of y.name is not correct"
assert sha1(str(y.name.lower()).encode("utf-8")+b"4d5b290dc60778af").hexdigest() == "a94cad89d9cc484c6896c6b5bb4c79fbc68b808e", "value of y.name is not correct"
assert sha1(str(y.name).encode("utf-8")+b"4d5b290dc60778af").hexdigest() == "a94cad89d9cc484c6896c6b5bb4c79fbc68b808e", "correct string value of y.name but incorrect case of letters"

assert sha1(str(type(fruit_fit is None)).encode("utf-8")+b"ea3498a55f6c786b").hexdigest() == "f0bc96535d161d9c7ec4228a5bf44a981c2d1828", "type of fruit_fit is None is not bool. fruit_fit is None should be a bool"
assert sha1(str(fruit_fit is None).encode("utf-8")+b"ea3498a55f6c786b").hexdigest() == "c7172d046204913b9063dbb97b03deae2068ae2b", "boolean value of fruit_fit is None is not correct"

assert sha1(str(type(type(fruit_fit))).encode("utf-8")+b"2afe0536d938b43a").hexdigest() == "14268ae005d54a7972dcc53bceecbd608988a1f2", "type of type(fruit_fit) is not correct"
assert sha1(str(type(fruit_fit)).encode("utf-8")+b"2afe0536d938b43a").hexdigest() == "95f3b8277d2f2b6b925352f1d5697cd7c03e5b9e", "value of type(fruit_fit) is not correct"

assert sha1(str(type(len(fruit_fit.named_steps))).encode("utf-8")+b"4d4545d37410263d").hexdigest() == "91c7385ac78512c8f43883e9dd13dbf617e56e17", "type of len(fruit_fit.named_steps) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(len(fruit_fit.named_steps)).encode("utf-8")+b"4d4545d37410263d").hexdigest() == "77d8991649d2260b23eeb47459cd7d6acfe04f5b", "value of len(fruit_fit.named_steps) is not correct"

assert sha1(str(type(fruit_fit.named_steps.keys())).encode("utf-8")+b"15284c8c8ef05b94").hexdigest() == "0aaab5e9fd0d210acf88317e070b77348815667a", "type of fruit_fit.named_steps.keys() is not correct"
assert sha1(str(fruit_fit.named_steps.keys()).encode("utf-8")+b"15284c8c8ef05b94").hexdigest() == "3f3039e0f47d691d396bbc238e240bceaf2d69a4", "value of fruit_fit.named_steps.keys() is not correct"

print('Success!')

**Question 2.3**
<br> {points: 1}

Now that we have created our K-nearest neighbor classifier object, let's predict the class labels for our test set.

First, pass your fitted model and the **test dataset** to the `predict` function. Then, use the `pd.concat` function to add the column of predictions to the original test data. 

*Assign your answer to an object called `fruit_test_predictions`.*

In [ ]:
# ___ = ___.predict(___[[___, ___]])
# ___ = pd.concat(
#     [
#         fruit_test.reset_index(drop=True),
#         pd.DataFrame(fruit_test_predictions, columns=["predicted"]),
#     ],
#     axis=1,
# ) # use concat to add the predictions column to the original test data

# your code here
raise NotImplementedError
fruit_test_predictions.head()

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_test_predictions is None)).encode("utf-8")+b"9925616a39ad04d1").hexdigest() == "4e1c192e0210e03645d374c18be510dfaff0a4a6", "type of fruit_test_predictions is None is not bool. fruit_test_predictions is None should be a bool"
assert sha1(str(fruit_test_predictions is None).encode("utf-8")+b"9925616a39ad04d1").hexdigest() == "8dd70efa27b73c38bb0b6a6b23971d06d9d919a1", "boolean value of fruit_test_predictions is None is not correct"

assert sha1(str(type(fruit_test_predictions)).encode("utf-8")+b"3a67f75da6db284a").hexdigest() == "bc17765162aa90694ed2b752d1357a2e232bf4d9", "type of type(fruit_test_predictions) is not correct"

assert sha1(str(type(fruit_test_predictions.shape)).encode("utf-8")+b"e7a1967ef3bbedc6").hexdigest() == "2a58c03b38bbd3b4a26537c357fe2e4da51b253e", "type of fruit_test_predictions.shape is not tuple. fruit_test_predictions.shape should be a tuple"
assert sha1(str(len(fruit_test_predictions.shape)).encode("utf-8")+b"e7a1967ef3bbedc6").hexdigest() == "e9494d50249da471ec931135b884e8ea3a810ca4", "length of fruit_test_predictions.shape is not correct"
assert sha1(str(sorted(map(str, fruit_test_predictions.shape))).encode("utf-8")+b"e7a1967ef3bbedc6").hexdigest() == "4c82c74b6ae867bdb5e8706bcdae7b9b6344ad1f", "values of fruit_test_predictions.shape are not correct"
assert sha1(str(fruit_test_predictions.shape).encode("utf-8")+b"e7a1967ef3bbedc6").hexdigest() == "d5fc103943cbc37ca30a99a7d221a210d2cf2849", "order of elements of fruit_test_predictions.shape is not correct"

assert sha1(str(type("predicted" in fruit_test_predictions.columns)).encode("utf-8")+b"b547f0807ad10d68").hexdigest() == "d86810c979724d39b3f6d239f053f35b34fb358c", "type of \"predicted\" in fruit_test_predictions.columns is not bool. \"predicted\" in fruit_test_predictions.columns should be a bool"
assert sha1(str("predicted" in fruit_test_predictions.columns).encode("utf-8")+b"b547f0807ad10d68").hexdigest() == "e90d213b2d7e6212552c17db009b78512731648f", "boolean value of \"predicted\" in fruit_test_predictions.columns is not correct"

print('Success!')

**Question 2.4**
<br> {points: 1}

Great! We have now computed some predictions for our test datasets! Wouldn't it be interesting if we could find out our classifier's accuracy? 

Thankfully, the `score` function from the `scikit-learn` package can help us. To get the statistics about the quality of our model, you need to call the `score` function on the `fruit_fit` model. Name the predictors as `X_test` and the target as `y_test`. We should pass the `X_test` and `y_test` into the `score` function.

*Assign your answer to an object called `fruit_prediction_accuracy`.*

In [ ]:
# ___ = ___[[___, ___]]
# ___ = ___["fruit_name"]

# ___ = fruit_fit.score(___, ___)

# your code here
raise NotImplementedError
fruit_prediction_accuracy

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_prediction_accuracy is None)).encode("utf-8")+b"a6554816c0886489").hexdigest() == "975419684970994a2eedc1ab602b296c4b120dae", "type of fruit_prediction_accuracy is None is not bool. fruit_prediction_accuracy is None should be a bool"
assert sha1(str(fruit_prediction_accuracy is None).encode("utf-8")+b"a6554816c0886489").hexdigest() == "048579198ddcf1fa2ce9b8a4ab71422c9887566a", "boolean value of fruit_prediction_accuracy is None is not correct"

assert sha1(str(type(fruit_prediction_accuracy)).encode("utf-8")+b"d2fd4e1a3e464b21").hexdigest() == "978312fee853339dace1eddebabb4a30febdf5d3", "type of fruit_prediction_accuracy is not correct"
assert sha1(str(fruit_prediction_accuracy).encode("utf-8")+b"d2fd4e1a3e464b21").hexdigest() == "c548bc2a3dafe343342c558e5b083fd3329e8aad", "value of fruit_prediction_accuracy is not correct"

print('Success!')

**Question 2.5**
<br> {points: 1}

Now, let's look at the *confusion matrix* for the classifier. This will show us the table of predicted labels and correct labels. 

A confusion matrix is essentially a classification matrix. The columns of the confusion matrix represent the actual class and the rows represent the predicted class (or vice versa). Shown below is an example of a confusion matrix.

|                  |          |  Actual Values |                |
|:----------------:|----------|:--------------:|:--------------:|
|                  |          |    Positive    |    Negative    |
|**Predicted Value**  | Positive |  True Positive | False Positive|
|                  | Negative | False Negative | True Negative  |


- A **true positive** is an outcome where the model correctly predicts the positive class.
- A **true negative** is an outcome where the model correctly predicts the negative class.
- A **false positive** is an outcome where the model incorrectly predicts the positive class.
- A **false negative** is an outcome where the model incorrectly predicts the negative class.

<br>

We can create a confusion matrix by using the `confusion_matrix` function from `scikit-learn` package. 

*Assign your answer to an object called `fruit_mat`*.

In [ ]:
# ___ = ___(
#     fruit_test_predictions[___],  # true labels
#     fruit_test_predictions[___],  # predicted labels
#     labels=fruit_fit.classes_, # specify the label for each class
# )

# your code here
raise NotImplementedError
fruit_mat

It is hard for us to interpret the confusion matrix as shown above. We could use the `ConfusionMatrixDisplay` function of the `scikit-learn` package to plot the confusion matrix. Please run the cell below.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(
    confusion_matrix=fruit_mat, display_labels=fruit_fit.classes_
)
disp.plot()

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_mat is None)).encode("utf-8")+b"24c4454f51609e4b").hexdigest() == "947d852711f28de880ce9950659d171b0cffc15f", "type of fruit_mat is None is not bool. fruit_mat is None should be a bool"
assert sha1(str(fruit_mat is None).encode("utf-8")+b"24c4454f51609e4b").hexdigest() == "7c5a7ede860c514260870fd1641f17d746e78b42", "boolean value of fruit_mat is None is not correct"

assert sha1(str(type(fruit_mat)).encode("utf-8")+b"443f3a7cbd57f201").hexdigest() == "eb2185f0ddb78f2af9bf87fc0df9d883636b94ce", "type of type(fruit_mat) is not correct"

assert sha1(str(type(fruit_mat.sum())).encode("utf-8")+b"dc4735585eaf09db").hexdigest() == "711473884215a2f6f9ea6d2b55dbd42e5256ec09", "type of fruit_mat.sum() is not correct"
assert sha1(str(fruit_mat.sum()).encode("utf-8")+b"dc4735585eaf09db").hexdigest() == "e274ffbefcbcdd078b9983dab5139eb67943ab84", "value of fruit_mat.sum() is not correct"

print('Success!')

**Question 2.6** Multiple Choice:
<br> {points: 1}

Reading `fruit_mat`, how many observations were labelled correctly?

A. 7

B. 8

C. 9

D. 14

*Assign your answer to an object called `answer2_6`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError
answer2_6

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer2_6)).encode("utf-8")+b"1f83313679f9c2f8").hexdigest() == "a66d621be607d8969ee6c1a26169330c8933e308", "type of answer2_6 is not str. answer2_6 should be an str"
assert sha1(str(len(answer2_6)).encode("utf-8")+b"1f83313679f9c2f8").hexdigest() == "1460c41aa7648ae87a3d789741aa16baf2de1ff5", "length of answer2_6 is not correct"
assert sha1(str(answer2_6.lower()).encode("utf-8")+b"1f83313679f9c2f8").hexdigest() == "7a1a21139b6406bf17428b415f7c3d62a5366a26", "value of answer2_6 is not correct"
assert sha1(str(answer2_6).encode("utf-8")+b"1f83313679f9c2f8").hexdigest() == "a076918f819a9ec42642d83004e3c85bdb82cddb", "correct string value of answer2_6 but incorrect case of letters"

print('Success!')

### 3. Cross-validation

**Question 3.1**
<br> {points: 1}

The vast majority of predictive models in statistics and machine learning have parameters that you have to pick. For the past few exercises, we have had to pick the number of neighbours for the class vote. But, is it possible to make this selection, *i.e., tune the model, in a principled way?* Ideally, we want to maximize the performance of our classifier on data *it hasn’t seen yet*.

There is also an important detail to mention about the process of tuning: we can, if we want to, split our overall training data up in multiple different ways, train and evaluate a classifier for each split, and then choose the parameter based on all of the different results. If we just split our overall training data once, our best parameter choice will depend strongly on whatever data was lucky enough to end up in the validation set. Perhaps using multiple different train / validation splits, we’ll get a better estimate of accuracy, which will lead to a better choice of the number of neighbours $K$ for the overall set of training data. 

This leads to the idea of cross-validation. In cross-validation, we split our overall training data into $C$ evenly-sized chunks, and then iteratively use 1 chunk as the validation set and combine the remaining $C−1$ chunks as the **training set.**

We can perform a cross-validation in Python using the `cross_validate` function from the `scikit-learn` package. The function returns a dictionary of float arrays of scores. To use this function, you have to identify model, the training set as well as specify the `cv` (the number of folds $C$, defaults to 5). We should set `return_train_score` to be `True` to return the training score as well. Before we use the function, we need to perform the pipeline analysis again. You can reuse the `fruit_preprocessor` and `knn_spec` objects you made earlier. Name your predictors as `X_val` and target as `y_val`.

*Assign your answer to an object called `fruit_vfold_score`*.

In [ ]:
np.random.seed(2020)  # DO NOT REMOVE

# ___ = ___[["mass", "color_score"]]
# ___ = ___["fruit_name"]
# ___ = ___(fruit_preprocessor, knn_spec)
# ___ = cross_validate(___,  ___, ___, return_train_score=True,)

# your code here
raise NotImplementedError
pd.DataFrame(fruit_vfold_score)

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_vfold_score is None)).encode("utf-8")+b"fa72cbdfc4d3203b").hexdigest() == "0ab6d8ef30a795fa846ffdb9123518c43a11452d", "type of fruit_vfold_score is None is not bool. fruit_vfold_score is None should be a bool"
assert sha1(str(fruit_vfold_score is None).encode("utf-8")+b"fa72cbdfc4d3203b").hexdigest() == "5cb35e22c428010fd03431f40a24e5932fca289a", "boolean value of fruit_vfold_score is None is not correct"

assert sha1(str(type(fruit_vfold_score)).encode("utf-8")+b"4d10a7cd3ed086e0").hexdigest() == "f014a79bb9086de9748ffa35a06bbd91ddfa17f0", "type of type(fruit_vfold_score) is not correct"

assert sha1(str(type(len(pd.DataFrame(fruit_vfold_score)))).encode("utf-8")+b"2e8cbbbdf9707aa9").hexdigest() == "ff8997db7a23ac502ff80c0eb2ad46a6d69e3b69", "type of len(pd.DataFrame(fruit_vfold_score)) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(len(pd.DataFrame(fruit_vfold_score))).encode("utf-8")+b"2e8cbbbdf9707aa9").hexdigest() == "134dac8962176577ad37ec3a752f492efd221b83", "value of len(pd.DataFrame(fruit_vfold_score)) is not correct"

assert sha1(str(type(pd.DataFrame(fruit_vfold_score).shape)).encode("utf-8")+b"88d101332e62d0c5").hexdigest() == "eec53ef5d99cf5912afb4dff05aefb7a19825689", "type of pd.DataFrame(fruit_vfold_score).shape is not tuple. pd.DataFrame(fruit_vfold_score).shape should be a tuple"
assert sha1(str(len(pd.DataFrame(fruit_vfold_score).shape)).encode("utf-8")+b"88d101332e62d0c5").hexdigest() == "4a00782b102d9857e464e1fb0ab7bd23b6b920bf", "length of pd.DataFrame(fruit_vfold_score).shape is not correct"
assert sha1(str(sorted(map(str, pd.DataFrame(fruit_vfold_score).shape))).encode("utf-8")+b"88d101332e62d0c5").hexdigest() == "7de357dc86127ac652f04562986b9f78c1ea5a38", "values of pd.DataFrame(fruit_vfold_score).shape are not correct"
assert sha1(str(pd.DataFrame(fruit_vfold_score).shape).encode("utf-8")+b"88d101332e62d0c5").hexdigest() == "08ebdfdca9f3c25dad4fe00c12a5389515858c39", "order of elements of pd.DataFrame(fruit_vfold_score).shape is not correct"

assert sha1(str(type(X_val.columns.values)).encode("utf-8")+b"a2bde23300fccbd4").hexdigest() == "0bca88929a518f60b1485034a779667f73609e59", "type of X_val.columns.values is not correct"
assert sha1(str(X_val.columns.values).encode("utf-8")+b"a2bde23300fccbd4").hexdigest() == "23e441029e3df3c94bda9c7a9e2601c68a79f185", "value of X_val.columns.values is not correct"

assert sha1(str(type(sum(X_val.color_score))).encode("utf-8")+b"baa18d3be5da40b2").hexdigest() == "5c09bbba5f0935a6d15b1c0c0195297ea9c387fb", "type of sum(X_val.color_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(X_val.color_score), 2)).encode("utf-8")+b"baa18d3be5da40b2").hexdigest() == "d046eadd93565102b12d5491a92d32a05d462815", "value of sum(X_val.color_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(X_val.mass))).encode("utf-8")+b"5d9acd2e7169e770").hexdigest() == "ade247dc7706b78a91e30de6675b6f8c193ffc9e", "type of sum(X_val.mass) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(X_val.mass)).encode("utf-8")+b"5d9acd2e7169e770").hexdigest() == "498c10cf2afa423b195ddfbdf4b8c82af76c7325", "value of sum(X_val.mass) is not correct"

print('Success!')

**Question 3.2**
<br> {points: 1}

Now that we have ran a cross-validation on each train/validation split, one has to ask, how accurate was the classifier's validation across the folds? We can aggregate the *mean* and *standard error* of these scores from each folds. The standard error is essentially a measure of how uncertain we are in the mean value. 

*Assign your answer to an object called `fruit_metrics_mean` and `fruit_metrics_std`.*

In [ ]:
# ___ = pd.DataFrame(___).___()
# ___ = pd.DataFrame(___).___()

# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_metrics_mean.shape)).encode("utf-8")+b"a82c06cce8e41075").hexdigest() == "2bcf633d9cb06382f8a612a201392b8081fe7546", "type of fruit_metrics_mean.shape is not tuple. fruit_metrics_mean.shape should be a tuple"
assert sha1(str(len(fruit_metrics_mean.shape)).encode("utf-8")+b"a82c06cce8e41075").hexdigest() == "b06a65ab4be9fb56792367b9ec18dc102a85b455", "length of fruit_metrics_mean.shape is not correct"
assert sha1(str(sorted(map(str, fruit_metrics_mean.shape))).encode("utf-8")+b"a82c06cce8e41075").hexdigest() == "2398a74f0eafa99be3a8e46d4de216c66f2289df", "values of fruit_metrics_mean.shape are not correct"
assert sha1(str(fruit_metrics_mean.shape).encode("utf-8")+b"a82c06cce8e41075").hexdigest() == "d9eba1891528f989ff78903fcfe1d9d69af0c9ba", "order of elements of fruit_metrics_mean.shape is not correct"

assert sha1(str(type(fruit_metrics_std.shape)).encode("utf-8")+b"1dad7ae2dfe8874a").hexdigest() == "8d0b3fb146479a02731c1ba76cfa7b183ff53eaf", "type of fruit_metrics_std.shape is not tuple. fruit_metrics_std.shape should be a tuple"
assert sha1(str(len(fruit_metrics_std.shape)).encode("utf-8")+b"1dad7ae2dfe8874a").hexdigest() == "556240a075ae7382f524264b512bb801d1eadb49", "length of fruit_metrics_std.shape is not correct"
assert sha1(str(sorted(map(str, fruit_metrics_std.shape))).encode("utf-8")+b"1dad7ae2dfe8874a").hexdigest() == "d10ef977e8012c491e6a238f70b83e7287a9a411", "values of fruit_metrics_std.shape are not correct"
assert sha1(str(fruit_metrics_std.shape).encode("utf-8")+b"1dad7ae2dfe8874a").hexdigest() == "dd70a751691e8575213a1fed8918b4a0f6ee7eb6", "order of elements of fruit_metrics_std.shape is not correct"

assert sha1(str(type(fruit_metrics_mean.train_score)).encode("utf-8")+b"3bdaef72fe2edaf3").hexdigest() == "5eb7719a2285782ee428cbce630357f821174f19", "type of fruit_metrics_mean.train_score is not correct"
assert sha1(str(fruit_metrics_mean.train_score).encode("utf-8")+b"3bdaef72fe2edaf3").hexdigest() == "9a311a1788cb35212a743df726258533f7d682a3", "value of fruit_metrics_mean.train_score is not correct"

assert sha1(str(type(fruit_metrics_std.train_score)).encode("utf-8")+b"1b15546ee9e84496").hexdigest() == "d555416bc931910ceb031faf1c563b7689b41393", "type of fruit_metrics_std.train_score is not correct"
assert sha1(str(fruit_metrics_std.train_score).encode("utf-8")+b"1b15546ee9e84496").hexdigest() == "21104f47a29ad1e7fd4d363d64289035ce1a5744", "value of fruit_metrics_std.train_score is not correct"

assert sha1(str(type(fruit_metrics_mean.test_score)).encode("utf-8")+b"7f4bddd5cd7d767f").hexdigest() == "f9e5a2e0fd1429deceeac2aedc65464ea775bb61", "type of fruit_metrics_mean.test_score is not correct"
assert sha1(str(fruit_metrics_mean.test_score).encode("utf-8")+b"7f4bddd5cd7d767f").hexdigest() == "7b8929e21e0f4fd47800ba17711d7fd460c82140", "value of fruit_metrics_mean.test_score is not correct"

assert sha1(str(type(fruit_metrics_std.test_score)).encode("utf-8")+b"9bb70b912f2de098").hexdigest() == "7c861b81fe3338e2eda117c3f84c58ec5ef8c38a", "type of fruit_metrics_std.test_score is not correct"
assert sha1(str(fruit_metrics_std.test_score).encode("utf-8")+b"9bb70b912f2de098").hexdigest() == "3e07a80076102c5580661f5991d48a691acae36a", "value of fruit_metrics_std.test_score is not correct"

print('Success!')

## 4. Parameter value selection

Using a 5-fold cross-validation, we have established a prediction accuracy for our classifier. 

If we had to improve our classifier, we have to change the parameter: number of neighbours, $K$. Since cross-validation helps us evaluate the accuracy of our classifier, we can use cross-validation to calculate an accuracy for each value of $K$ in a reasonable range, and then pick the value of $K$ that gives us the best accuracy. 

The great thing about the `scikit-learn` package is that it provides two following build-in methods for tuning parameters. Each parameter in the model can be adjusted rather than given a specific value. We can define a set of values for each hyperparamters and find the best parameters in this set.

- Exhaustive grid search
    - [sklearn.model_selection.GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
    - A user specifies a set of values for each hyperparameter.
    - The method considers product of the sets and then evaluates each combination one by one.
    
- Randomized hyperparameter optimization
    - [sklearn.model_selection.RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
    - Samples configurations at random until certain budget (e.g., time) is exhausted

**Question 4.0**
<br> {points: 1}

Create a new K-nearest neighbor model specification but instead of specifying a particular value for the `n_neighbors` argument, try `GridSearchCV` and `RandomizedSearchCV`. Before we use `GridSearchCV` and `RandomizedSearchCV` to run 4-fold cross-validations to tune hyperparameters, we should define the parameter grid by passing the set of values for each parameters that you would like to tune. We would also need to redefine the pipeline to use default values for parameters. 

*Assign your answer to an object called `knn_tune_grid` and `knn_tune_random` respectively.* 

In [ ]:
### Run this cell
param_grid = {
    "kneighborsclassifier__n_neighbors": range(2, 15, 1),
}
fruit_tune_pipe = make_pipeline(fruit_preprocessor, KNeighborsClassifier())

In [ ]:
# ___ = GridSearchCV(
#     ___, ___, cv=__,
# )
# ___ = RandomizedSearchCV(
#     ___, ___, n_iter=10, cv=_, random_state=2023
# )

# your code here
raise NotImplementedError
knn_tune_grid
knn_tune_random

In [ ]:
from hashlib import sha1
assert sha1(str(type(knn_tune_grid is None)).encode("utf-8")+b"1ef4f75220a2d9e4").hexdigest() == "699106ce653f14014be2f304616ba695cd35b595", "type of knn_tune_grid is None is not bool. knn_tune_grid is None should be a bool"
assert sha1(str(knn_tune_grid is None).encode("utf-8")+b"1ef4f75220a2d9e4").hexdigest() == "1c74d1f9330b1a3b2d00225a539c0347a72dc065", "boolean value of knn_tune_grid is None is not correct"

assert sha1(str(type(type(knn_tune_grid))).encode("utf-8")+b"e469241153721ba3").hexdigest() == "eb9bf3630e1968e1cd3f503d644d1ab3402ac85a", "type of type(knn_tune_grid) is not correct"
assert sha1(str(type(knn_tune_grid)).encode("utf-8")+b"e469241153721ba3").hexdigest() == "19eaf1378ed02fb808f3c754d7af1f13bbd89045", "value of type(knn_tune_grid) is not correct"

assert sha1(str(type(knn_tune_grid.param_grid.keys())).encode("utf-8")+b"1aed05cd78b7ee92").hexdigest() == "7302569417e666881abc992956d498d38fcfdd8e", "type of knn_tune_grid.param_grid.keys() is not correct"
assert sha1(str(knn_tune_grid.param_grid.keys()).encode("utf-8")+b"1aed05cd78b7ee92").hexdigest() == "8dd44d183603e3fe1093f50c9b9545db10edba87", "value of knn_tune_grid.param_grid.keys() is not correct"

assert sha1(str(type(knn_tune_grid.estimator.named_steps.keys())).encode("utf-8")+b"9d88b6b8f59de285").hexdigest() == "e06930f3855c760a262372c51f4fac0086bb8821", "type of knn_tune_grid.estimator.named_steps.keys() is not correct"
assert sha1(str(knn_tune_grid.estimator.named_steps.keys()).encode("utf-8")+b"9d88b6b8f59de285").hexdigest() == "2bc431d5c300fb944afcbf34e1bce8e92aa0206a", "value of knn_tune_grid.estimator.named_steps.keys() is not correct"

assert sha1(str(type(knn_tune_random is None)).encode("utf-8")+b"f793e23ffc2c8216").hexdigest() == "a0e2fcf6a2c4aca8c664edc39f9d7d757b8a98c0", "type of knn_tune_random is None is not bool. knn_tune_random is None should be a bool"
assert sha1(str(knn_tune_random is None).encode("utf-8")+b"f793e23ffc2c8216").hexdigest() == "3c694a57f4dd7b0899bfc047c259a405784228dc", "boolean value of knn_tune_random is None is not correct"

assert sha1(str(type(type(knn_tune_random))).encode("utf-8")+b"716298e39233b09b").hexdigest() == "a011a275ea059022351d64ff22f7147b1b773866", "type of type(knn_tune_random) is not correct"
assert sha1(str(type(knn_tune_random)).encode("utf-8")+b"716298e39233b09b").hexdigest() == "162c536394a2180ab8558f48f0defbfed6919f49", "value of type(knn_tune_random) is not correct"

assert sha1(str(type(knn_tune_random.param_distributions.keys())).encode("utf-8")+b"8aa2f9d23760866d").hexdigest() == "b5111cba7e3990a22c2fc2b4529040a854b86492", "type of knn_tune_random.param_distributions.keys() is not correct"
assert sha1(str(knn_tune_random.param_distributions.keys()).encode("utf-8")+b"8aa2f9d23760866d").hexdigest() == "4ec4430270c3368e6e93f907b80b519a612e7b02", "value of knn_tune_random.param_distributions.keys() is not correct"

assert sha1(str(type(knn_tune_random.estimator.named_steps.keys())).encode("utf-8")+b"7158b3e955d7475d").hexdigest() == "57a50a9d45eb96f9214009ac030b8c4587a9c468", "type of knn_tune_random.estimator.named_steps.keys() is not correct"
assert sha1(str(knn_tune_random.estimator.named_steps.keys()).encode("utf-8")+b"7158b3e955d7475d").hexdigest() == "fa1cea62562827496c7d664826100e4c1bc6ed6a", "value of knn_tune_random.estimator.named_steps.keys() is not correct"

print('Success!')

**Question 4.1**
<br>{points: 1}

Now, let's fit the models to the data. Name the predictors as `X_tune` and target as `y_tune`.

*Assign your tuned models to objects called `knn_model_grid` and `knn_model_random`.*

Next, from `knn_model_*`, find out the `cv_results_` and save it in a dataframe. 

*Assign your answer to an object called `accuracies_grid` and `accuracies_random` respectively.*

In [ ]:
# ___ = ___[[___, ___]]
# ___ = ___[___]

# ___ = ___.fit(___, ___)
# ___ = ___.fit(___, ___)

# ___ = pd.DataFrame(___.cv_results_)
# ___ = pd.DataFrame(___.cv_results_)

# your code here
raise NotImplementedError
knn_model_grid
knn_model_random
accuracies_grid
accuracies_random

In [ ]:
from hashlib import sha1
assert sha1(str(type(type(knn_model_grid))).encode("utf-8")+b"cc7260147b9f739a").hexdigest() == "ae740582d45119e545309dd39186296db8de1b0d", "type of type(knn_model_grid) is not correct"
assert sha1(str(type(knn_model_grid)).encode("utf-8")+b"cc7260147b9f739a").hexdigest() == "dc8b2ad690753c51c6788ae909cadceab78329b7", "value of type(knn_model_grid) is not correct"

assert sha1(str(type(type(knn_model_random))).encode("utf-8")+b"156e808b94de8a49").hexdigest() == "cf6789c269aea0ba4c324c837259c8cbac2970c6", "type of type(knn_model_random) is not correct"
assert sha1(str(type(knn_model_random)).encode("utf-8")+b"156e808b94de8a49").hexdigest() == "aa6cc0d6c7f6d31adf7add6c4c2307eb59a59647", "value of type(knn_model_random) is not correct"

assert sha1(str(type(accuracies_grid is None)).encode("utf-8")+b"d878a0faebe308cd").hexdigest() == "c259bbcae00cd8de275648bc97ee5729f093e202", "type of accuracies_grid is None is not bool. accuracies_grid is None should be a bool"
assert sha1(str(accuracies_grid is None).encode("utf-8")+b"d878a0faebe308cd").hexdigest() == "9eba324a302a3a40e6f98e7b482c841d1f1ab6af", "boolean value of accuracies_grid is None is not correct"

assert sha1(str(type(accuracies_grid)).encode("utf-8")+b"1725045e91f163dc").hexdigest() == "6eebe25aa3b73c673f857662981c5e2978f70681", "type of type(accuracies_grid) is not correct"

assert sha1(str(type(accuracies_grid.shape)).encode("utf-8")+b"17e5b54ddff545cc").hexdigest() == "0818eea2397bb7d7be9d9706577e55decc65b867", "type of accuracies_grid.shape is not tuple. accuracies_grid.shape should be a tuple"
assert sha1(str(len(accuracies_grid.shape)).encode("utf-8")+b"17e5b54ddff545cc").hexdigest() == "7f1e0fc3b6a4c1df41378024bd214f2489952b86", "length of accuracies_grid.shape is not correct"
assert sha1(str(sorted(map(str, accuracies_grid.shape))).encode("utf-8")+b"17e5b54ddff545cc").hexdigest() == "58b205e6654eb8a1a00fb60c935808c3a7af6792", "values of accuracies_grid.shape are not correct"
assert sha1(str(accuracies_grid.shape).encode("utf-8")+b"17e5b54ddff545cc").hexdigest() == "bea10675c67d644261f65fb1db4794fce4173d9f", "order of elements of accuracies_grid.shape is not correct"

assert sha1(str(type(sum(accuracies_grid.mean_test_score))).encode("utf-8")+b"f261eaafa5fad2f1").hexdigest() == "8c6c228b1d7b75a2f1f5199016f9c432189cade2", "type of sum(accuracies_grid.mean_test_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(accuracies_grid.mean_test_score), 2)).encode("utf-8")+b"f261eaafa5fad2f1").hexdigest() == "9b8f2ba8c1a9007c8025dc0965bdb7babe96c976", "value of sum(accuracies_grid.mean_test_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(accuracies_grid.std_test_score))).encode("utf-8")+b"bba74d9c2c254c77").hexdigest() == "89e940c61841b14339852fd329020d78bc341984", "type of sum(accuracies_grid.std_test_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(accuracies_grid.std_test_score), 2)).encode("utf-8")+b"bba74d9c2c254c77").hexdigest() == "d31cf39478dfb92f3aa2852d40aef53f877a3265", "value of sum(accuracies_grid.std_test_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(accuracies_grid.param_kneighborsclassifier__n_neighbors))).encode("utf-8")+b"3df6260e0906d2e9").hexdigest() == "4c6ba0213387e47799a2806afbdf6cd900ef4bc3", "type of sum(accuracies_grid.param_kneighborsclassifier__n_neighbors) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(accuracies_grid.param_kneighborsclassifier__n_neighbors)).encode("utf-8")+b"3df6260e0906d2e9").hexdigest() == "091ee9551b9caf2fe8a1c73598dfee4eacd612f2", "value of sum(accuracies_grid.param_kneighborsclassifier__n_neighbors) is not correct"

assert sha1(str(type(accuracies_random is None)).encode("utf-8")+b"e267c4d86d7a3e52").hexdigest() == "60a43cc99060f3795d83e798e65894aa9bcbad39", "type of accuracies_random is None is not bool. accuracies_random is None should be a bool"
assert sha1(str(accuracies_random is None).encode("utf-8")+b"e267c4d86d7a3e52").hexdigest() == "de10f981c3814fa7b302ef074cdbb72b5118c4a3", "boolean value of accuracies_random is None is not correct"

assert sha1(str(type(accuracies_random)).encode("utf-8")+b"8cadf09da4078b45").hexdigest() == "7eae0376c5a9df1a92fb1c2c802eb2cc5e670c82", "type of type(accuracies_random) is not correct"

assert sha1(str(type(accuracies_random.shape)).encode("utf-8")+b"dfc7562d9fd05487").hexdigest() == "db285b0718db9805a85ad556cc7113090916c02f", "type of accuracies_random.shape is not tuple. accuracies_random.shape should be a tuple"
assert sha1(str(len(accuracies_random.shape)).encode("utf-8")+b"dfc7562d9fd05487").hexdigest() == "6e5417cb76b279d7a8a89bd2c3a99cfba99a476c", "length of accuracies_random.shape is not correct"
assert sha1(str(sorted(map(str, accuracies_random.shape))).encode("utf-8")+b"dfc7562d9fd05487").hexdigest() == "214f4480a472a137ede7b4a79a8ac285afcaf344", "values of accuracies_random.shape are not correct"
assert sha1(str(accuracies_random.shape).encode("utf-8")+b"dfc7562d9fd05487").hexdigest() == "d775279d00310ee2d84fc951cdc9a4fdadcef82f", "order of elements of accuracies_random.shape is not correct"

assert sha1(str(type(sum(accuracies_random.mean_test_score))).encode("utf-8")+b"2cf01fc7f7ecc68d").hexdigest() == "ff0d7a7adb6bc3c9ae25f1cd1a41f53dc02de74b", "type of sum(accuracies_random.mean_test_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(accuracies_random.mean_test_score), 2)).encode("utf-8")+b"2cf01fc7f7ecc68d").hexdigest() == "f37f843bdb772370073429dc183469f0e25ce2dc", "value of sum(accuracies_random.mean_test_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(accuracies_random.std_test_score))).encode("utf-8")+b"956480af890b1cc3").hexdigest() == "34276a1b2c3eabb1e436c23b18a9d2b42afb5f3f", "type of sum(accuracies_random.std_test_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(accuracies_random.std_test_score), 2)).encode("utf-8")+b"956480af890b1cc3").hexdigest() == "a9e61699e240e3905dc9d2436414769b3eeb93c4", "value of sum(accuracies_random.std_test_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(accuracies_random.param_kneighborsclassifier__n_neighbors))).encode("utf-8")+b"9886992b1caeeb05").hexdigest() == "71039c56a38c23a8ac68ad52282e508cd4f3ce67", "type of sum(accuracies_random.param_kneighborsclassifier__n_neighbors) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(accuracies_random.param_kneighborsclassifier__n_neighbors)).encode("utf-8")+b"9886992b1caeeb05").hexdigest() == "f2cd4cb0a6b481e2add62c16100c9395f41bd6cc", "value of sum(accuracies_random.param_kneighborsclassifier__n_neighbors) is not correct"

print('Success!')

**Question 4.2**
<br>{points: 1} 


Now, let's find the best value of the number of neighbors. 

Create a line plot using the `accuracies_*` dataframes with `param_kneighborsclassifier__n_neighbors` on the x-axis and the `mean_test_score` on the y-axis. 

*Assign your answer to an object called `accuracy_versus_k_grid` and `accuracy_versus_k_random`.*

In [ ]:
# ___ = (
#     alt.Chart(___, title="Grid Search")
#     .mark_line(point=True)
#     .encode(
#         x=alt.X(
#             ___,
#             title="Neighbors",
#             scale=alt.Scale(zero=False),
#         ),
#         y=alt.Y(
#             ___, 
#             title="Mean Test Score", 
#             scale=alt.Scale(zero=False)
#         ),
#     )
#     .configure_axis(labelFontSize=10, titleFontSize=15)
#     .properties(width=400, height=300)
# )

# ___ = (
#     alt.Chart(___, title="Randomized Search")
#     .mark_line(point=True)
#     .encode(
#         x=alt.X(
#             ___,
#             title="Neighbors",
#             scale=alt.Scale(zero=False),
#         ),
#         y=alt.Y(
#             ___, 
#             title="Mean Test Score", 
#             scale=alt.Scale(zero=False)
#         ),
#     )
#     .configure_axis(labelFontSize=10, titleFontSize=15)
#     .properties(width=400, height=300)
# )


# your code here
raise NotImplementedError

In [ ]:
# run the cell
accuracy_versus_k_grid

In [ ]:
# run the cell
accuracy_versus_k_random

In [ ]:
from hashlib import sha1
assert sha1(str(type(accuracy_versus_k_grid is None)).encode("utf-8")+b"93308ebcb925cb87").hexdigest() == "b61dcce54ed217642e45d6a21ae582c35fa0a2e1", "type of accuracy_versus_k_grid is None is not bool. accuracy_versus_k_grid is None should be a bool"
assert sha1(str(accuracy_versus_k_grid is None).encode("utf-8")+b"93308ebcb925cb87").hexdigest() == "c8bf479ec48ecf2f35d40ea70c4f88997f1d281f", "boolean value of accuracy_versus_k_grid is None is not correct"

assert sha1(str(type(accuracy_versus_k_grid.encoding.x.field)).encode("utf-8")+b"266b7b7315e1ba9c").hexdigest() == "70c6847d8f93e8986629f9f84a766faa101e0a14", "type of accuracy_versus_k_grid.encoding.x.field is not str. accuracy_versus_k_grid.encoding.x.field should be an str"
assert sha1(str(len(accuracy_versus_k_grid.encoding.x.field)).encode("utf-8")+b"266b7b7315e1ba9c").hexdigest() == "42c09a99acbc44b1081b9d7ec1efbedbd6f3e258", "length of accuracy_versus_k_grid.encoding.x.field is not correct"
assert sha1(str(accuracy_versus_k_grid.encoding.x.field.lower()).encode("utf-8")+b"266b7b7315e1ba9c").hexdigest() == "6f3b53287601590332a7fdb0c388bc33a49cac6f", "value of accuracy_versus_k_grid.encoding.x.field is not correct"
assert sha1(str(accuracy_versus_k_grid.encoding.x.field).encode("utf-8")+b"266b7b7315e1ba9c").hexdigest() == "6f3b53287601590332a7fdb0c388bc33a49cac6f", "correct string value of accuracy_versus_k_grid.encoding.x.field but incorrect case of letters"

assert sha1(str(type(accuracy_versus_k_grid.encoding.y.field)).encode("utf-8")+b"0303f62a2b8b5aef").hexdigest() == "c15c611e36b8bf58a53235c344a5e1db5b8bbdfd", "type of accuracy_versus_k_grid.encoding.y.field is not str. accuracy_versus_k_grid.encoding.y.field should be an str"
assert sha1(str(len(accuracy_versus_k_grid.encoding.y.field)).encode("utf-8")+b"0303f62a2b8b5aef").hexdigest() == "f0c783a0ed0a966db332d5bb1861324fc3de282d", "length of accuracy_versus_k_grid.encoding.y.field is not correct"
assert sha1(str(accuracy_versus_k_grid.encoding.y.field.lower()).encode("utf-8")+b"0303f62a2b8b5aef").hexdigest() == "158b7b95d171a0cb30c1cf9a9feb6d66d15fde6d", "value of accuracy_versus_k_grid.encoding.y.field is not correct"
assert sha1(str(accuracy_versus_k_grid.encoding.y.field).encode("utf-8")+b"0303f62a2b8b5aef").hexdigest() == "158b7b95d171a0cb30c1cf9a9feb6d66d15fde6d", "correct string value of accuracy_versus_k_grid.encoding.y.field but incorrect case of letters"

assert sha1(str(type(accuracy_versus_k_grid.mark.type)).encode("utf-8")+b"2ba2ec6571bfa47c").hexdigest() == "166feae2cae0e9fbb3569c53c1eab896a593aea2", "type of accuracy_versus_k_grid.mark.type is not str. accuracy_versus_k_grid.mark.type should be an str"
assert sha1(str(len(accuracy_versus_k_grid.mark.type)).encode("utf-8")+b"2ba2ec6571bfa47c").hexdigest() == "3ecf6788905b6706ea51395a33de48a5f9ea5364", "length of accuracy_versus_k_grid.mark.type is not correct"
assert sha1(str(accuracy_versus_k_grid.mark.type.lower()).encode("utf-8")+b"2ba2ec6571bfa47c").hexdigest() == "398f6291357690ea53a04391606e0f49ec80374b", "value of accuracy_versus_k_grid.mark.type is not correct"
assert sha1(str(accuracy_versus_k_grid.mark.type).encode("utf-8")+b"2ba2ec6571bfa47c").hexdigest() == "398f6291357690ea53a04391606e0f49ec80374b", "correct string value of accuracy_versus_k_grid.mark.type but incorrect case of letters"

assert sha1(str(type(accuracy_versus_k_grid.mark['point'])).encode("utf-8")+b"d6bffbf6a1391c82").hexdigest() == "02c8330ee3c8b411d1be1c710200e16fb41ec0df", "type of accuracy_versus_k_grid.mark['point'] is not bool. accuracy_versus_k_grid.mark['point'] should be a bool"
assert sha1(str(accuracy_versus_k_grid.mark['point']).encode("utf-8")+b"d6bffbf6a1391c82").hexdigest() == "ec31cc5f757e4b465ef33ee62c0b0f9845ff169b", "boolean value of accuracy_versus_k_grid.mark['point'] is not correct"

assert sha1(str(type(accuracy_versus_k_random is None)).encode("utf-8")+b"0ce3f4fed399c494").hexdigest() == "2ae0e4bf4768b8be17621daaf8c9817a148c65d6", "type of accuracy_versus_k_random is None is not bool. accuracy_versus_k_random is None should be a bool"
assert sha1(str(accuracy_versus_k_random is None).encode("utf-8")+b"0ce3f4fed399c494").hexdigest() == "735d872b3a4119cae88e26f18d33e93e975f6b70", "boolean value of accuracy_versus_k_random is None is not correct"

assert sha1(str(type(accuracy_versus_k_random.encoding.x.field)).encode("utf-8")+b"10e862d68a226f17").hexdigest() == "5690016097820b470d1e8f275ff1db1b7a3d33d6", "type of accuracy_versus_k_random.encoding.x.field is not str. accuracy_versus_k_random.encoding.x.field should be an str"
assert sha1(str(len(accuracy_versus_k_random.encoding.x.field)).encode("utf-8")+b"10e862d68a226f17").hexdigest() == "27140eb08e666d26805342751e601568f87fc8f5", "length of accuracy_versus_k_random.encoding.x.field is not correct"
assert sha1(str(accuracy_versus_k_random.encoding.x.field.lower()).encode("utf-8")+b"10e862d68a226f17").hexdigest() == "bc10a4541ae97eaea7d4f4209249817d46b4cf32", "value of accuracy_versus_k_random.encoding.x.field is not correct"
assert sha1(str(accuracy_versus_k_random.encoding.x.field).encode("utf-8")+b"10e862d68a226f17").hexdigest() == "bc10a4541ae97eaea7d4f4209249817d46b4cf32", "correct string value of accuracy_versus_k_random.encoding.x.field but incorrect case of letters"

assert sha1(str(type(accuracy_versus_k_random.encoding.y.field)).encode("utf-8")+b"172a3b698242ee74").hexdigest() == "d5718e37a9d4aeec8c56c65ffb73f6aa4e233118", "type of accuracy_versus_k_random.encoding.y.field is not str. accuracy_versus_k_random.encoding.y.field should be an str"
assert sha1(str(len(accuracy_versus_k_random.encoding.y.field)).encode("utf-8")+b"172a3b698242ee74").hexdigest() == "60b98f4e127238e9f98cf8293a8ea51ed7fc1f4c", "length of accuracy_versus_k_random.encoding.y.field is not correct"
assert sha1(str(accuracy_versus_k_random.encoding.y.field.lower()).encode("utf-8")+b"172a3b698242ee74").hexdigest() == "aa72c4f25c8c6966f246cb1de6db294018467b3b", "value of accuracy_versus_k_random.encoding.y.field is not correct"
assert sha1(str(accuracy_versus_k_random.encoding.y.field).encode("utf-8")+b"172a3b698242ee74").hexdigest() == "aa72c4f25c8c6966f246cb1de6db294018467b3b", "correct string value of accuracy_versus_k_random.encoding.y.field but incorrect case of letters"

assert sha1(str(type(accuracy_versus_k_random.mark.type)).encode("utf-8")+b"ace4d10c09d83989").hexdigest() == "71831603c88e0235932e476dff488ba60609ef3b", "type of accuracy_versus_k_random.mark.type is not str. accuracy_versus_k_random.mark.type should be an str"
assert sha1(str(len(accuracy_versus_k_random.mark.type)).encode("utf-8")+b"ace4d10c09d83989").hexdigest() == "8877e8fbf0197da1add7b505980af52acf0788ca", "length of accuracy_versus_k_random.mark.type is not correct"
assert sha1(str(accuracy_versus_k_random.mark.type.lower()).encode("utf-8")+b"ace4d10c09d83989").hexdigest() == "ee8bc8733883e0ca7218addf2455eba099cea22d", "value of accuracy_versus_k_random.mark.type is not correct"
assert sha1(str(accuracy_versus_k_random.mark.type).encode("utf-8")+b"ace4d10c09d83989").hexdigest() == "ee8bc8733883e0ca7218addf2455eba099cea22d", "correct string value of accuracy_versus_k_random.mark.type but incorrect case of letters"

assert sha1(str(type(accuracy_versus_k_random.mark['point'])).encode("utf-8")+b"e1b258980db159b6").hexdigest() == "baad3483d155eef81464ebf2153da2375ba52bcc", "type of accuracy_versus_k_random.mark['point'] is not bool. accuracy_versus_k_random.mark['point'] should be a bool"
assert sha1(str(accuracy_versus_k_random.mark['point']).encode("utf-8")+b"e1b258980db159b6").hexdigest() == "9d9a1a9934e12a8afdb6d1fc58a34fd4ba0cb4cd", "boolean value of accuracy_versus_k_random.mark['point'] is not correct"

print('Success!')

From the plots above, we can see that $K = 2$, $3$, or $4$ provides the highest accuracy. Larger $K$ values result in a reduced accuracy estimate. Remember: the values you see on this plot are estimates of the true accuracy of our classifier. Although the $K = 2$, $3$ or $4$ value is higher than the others on this plot, that doesn’t mean the classifier is necessarily more accurate with this parameter value! 

Great, now you have completed a full analysis with cross-validation using the `scikit-learn` package! For your information, we can choose any number of folds and typically, the more we use the better our accuracy estimate will be (lower standard error). However, more folds would mean a greater computation time. In practice, $cv$ is chosen to be either 5 or 10. 